# Topic Modeling on Bukowski's Poems

On the first post about Bukowski's poems we explored the top words and their polarity. From inspection these groups seemed to be associated to 4 main topics, which also happen to be mentioned on the writter's legacy website. It would be interesting to see if these same topics show up when applicating a generative statistical modeling, such as the Latent Dirichlet Allocation (LDA). To do this and visualize the results I'll use the pyLDAvis and scikit-learn packages.

In [187]:
## Setup and importing libraries.
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning, module='.*/IPython/.*')

from tqdm import tqdm
import pandas as pd
import gensim
import pyLDAvis
import pyLDAvis.sklearn

from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer 

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

pyLDAvis.enable_notebook()

In [3]:
## Load the pickle file.
content = pd.read_pickle('data/content.pkl')

## Add each poem as an element of a list.
raw_poems = []
for p in content.poem.map(int).unique():
    flat_poem = ' '.join([text for text in content.loc[content.poem == p, 'text']])
    raw_poems.append(flat_poem)

In order to perform the LDA, we need to generate a document-term matrix representation of all the columns. Rows in our matrix represent words, while columns correspond to different words. The `CountVectorizer` function helps us do this, and it also performs some pre-processing (removing stop-words, generating bigrams, setting minimum word frequencies, setting everything to lowercase).

In [325]:
## Generate sparse document-term matrix.
dtm_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                ngram_range = (1,2), 
                                min_df = 20,
                                max_df = 1.0)
dtm_poems = dtm_vectorizer.fit_transform(raw_poems)
print dtm_poems.shape

(1363, 1035)


## Fitting and Visualizing the LDA Model
To fit the LDA algorithm, we are required to set the number of topics it will associate the words with. This is an arbitrary decision, and in this case I'll start with 7.

In [252]:
lda_model = LatentDirichletAllocation(n_topics=7)
lda_model.fit(dtm_poems)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_jobs=1, n_topics=7, perp_tol=0.1, random_state=None,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [253]:
pyLDAvis.sklearn.prepare(lda_model, dtm_poems, dtm_vectorizer, mds='tsne')

PreparedData(topic_coordinates=            Freq  cluster  topics           x           y
topic                                                    
2      37.173234        1       1  -28.836723   68.381833
5      31.951688        1       2   98.938224  -12.339956
4      16.202847        1       3   73.891627 -131.253180
1       8.231556        1       4   57.464121  157.834842
6       4.041853        1       5  -49.528391 -156.612582
3       1.674322        1       6 -113.418121  -39.681155
0       0.724499        1       7   -5.140661  -48.631170, topic_info=     Category         Freq          Term        Total  loglift  logprob
term                                                                   
1490  Default  1159.000000          said  1159.000000  30.0000  30.0000
116   Default   257.000000          beer   257.000000  29.0000  29.0000
731   Default   845.000000           got   845.000000  28.0000  28.0000
1922  Default   389.000000        walked   389.000000  27.0000  27.0000
1010  Default  1531.000000          like  1531.000000  26.0000  26.0000
1169  Default   133.000000         music   133.000000  25.0000  25.0000
253   Default   360.000000          came   360.000000  24.0000  24.0000
1968  Default   398.000000          went   398.000000  23.0000  23.0000
2015  Default   190.000000          work   190.000000  22.0000  22.0000
414   Default   159.000000          days   159.000000  21.0000  21.0000
1049  Default   255.000000        looked   255.000000  20.0000  20.0000
1278  Default   415.000000        people   415.000000  19.0000  19.0000
1091  Default   670.000000           man   670.000000  18.0000  18.0000
508   Default   213.000000         drunk   213.000000  17.0000  17.0000
57    Default   173.000000           ask   173.000000  16.0000  16.0000
58    Default   264.000000         asked   264.000000  15.0000  15.0000
185   Default   128.000000        bottle   128.000000  14.0000  14.0000
1511  Default   250.000000           sat   250.000000  13.0000  13.0000
416   Default   312.000000          dead   312.000000  12.0000  12.0000
1519  Default   214.000000          says   214.000000  11.0000  11.0000
258   Default   335.000000           car   335.000000  10.0000  10.0000
1064  Default   360.000000          love   360.000000   9.0000   9.0000
1515  Default   193.000000           saw   193.000000   8.0000   8.0000
1192  Default   441.000000         night   441.000000   7.0000   7.0000
1397  Default   146.000000         radio   146.000000   6.0000   6.0000
96    Default   147.000000           bar   147.000000   5.0000   5.0000
1217  Default   625.000000           old   625.000000   4.0000   4.0000
1722  Default   144.000000         stood   144.000000   3.0000   3.0000
494   Default   159.000000      drinking   159.000000   2.0000   2.0000
1958  Default   369.000000           way   369.000000   1.0000   1.0000
...       ...          ...           ...          ...      ...      ...
1200   Topic7     2.857974        normal    12.953063   3.4162  -5.5106
1165   Topic7     5.056466        mozart    24.376483   3.3545  -4.9401
1612   Topic7     4.012023        sister    19.757188   3.3332  -5.1715
355    Topic7     4.003321   continually    21.124704   3.2641  -5.1736
1142   Topic7     2.389962        monica    13.365096   3.2061  -5.6895
1510   Topic7     2.330147  santa monica    13.370870   3.1803  -5.7148
1173   Topic7     1.837895         named    10.712177   3.1647  -5.9521
745    Topic7     2.363091    grandstand    14.047929   3.1449  -5.7008
356    Topic7     2.734454      continue    17.665094   3.0618  -5.5548
222    Topic7     1.905773     buildings    12.645681   3.0350  -5.9159
2044   Topic7     2.026749     yeah yeah    13.768443   3.0115  -5.8543
2      Topic7     2.576059      accepted    18.009281   2.9828  -5.6145
1169   Topic7    18.590247         music   133.582764   2.9554  -3.6381
2017   Topic7     1.921204       workers    14.629095   2.8974  -5.9078
845    Topic7     2.721929      horrible    20.9793

Examining each of the word clusters, or topics, we see some very general ones and others having too much noise. However there are a couple of interesting ones:

- **Cluster 3:** General topic seems to be around love and dead. Other related words are lives, laughter, reason and madness.
- **Cluster 5:** Horse tracks. The author's visits to this place are also a common subject in his writtings, and this gets reflected here.
- **Cluster 6:** Alcohol and its multiple manifestations.
- **Cluster 7:** Music and art, which are also commonly present on Bukowski's life.

## Expanding Number of Topics
If we define a larger number of clusters for the LDA model, these will tend to become more granular, particularly for the smaller ones. Below I redo the exercise but for 25 groups, I won't go into details but the user is encouraged to explore. 

In [341]:
lda_model = LatentDirichletAllocation(n_topics=25)
lda_model.fit(dtm_poems)
pyLDAvis.sklearn.prepare(lda_model, dtm_poems, dtm_vectorizer, R=20)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
13     24.664318        1       1 -0.074051  0.053798
11     23.229509        1       2 -0.064008  0.164120
23     13.975983        1       3 -0.110824  0.023524
17      5.994711        1       4 -0.103482  0.039258
19      4.588245        1       5 -0.050417  0.175548
16      3.967552        1       6 -0.095725  0.146836
12      3.691477        1       7 -0.157568  0.036788
7       3.474120        1       8 -0.071607 -0.058344
0       3.381880        1       9 -0.116777 -0.096513
24      2.534305        1      10 -0.113686 -0.139155
6       2.319543        1      11 -0.154014 -0.115928
15      2.103734        1      12 -0.057420  0.112983
2       1.034710        1      13  0.014564 -0.159526
10      0.838749        1      14 -0.091621 -0.140314
9       0.741720        1      15  0.077131  0.084599
4       0.718235        1      16  0.015501 -0.144745
5       0.493983        1      17  0.079507 -0.032183
8       0.460002        1      18  0.215517 -0.068103
20      0.369913        1      19  0.050535  0.046696
1       0.341854        1      20  0.105145 -0.079326
18      0.294118        1      21  0.200692  0.020006
14      0.281115        1      22  0.148753  0.015397
22      0.277816        1      23  0.195967  0.009862
3       0.157482        1      24  0.106849  0.036665
21      0.064927        1      25  0.051041  0.068056, topic_info=     Category         Freq          Term        Total  loglift  logprob
term                                                                   
499   Default  1551.000000          like  1551.000000  15.0000  15.0000
549   Default   670.000000           man   670.000000  14.0000  14.0000
745   Default  1174.000000          said  1174.000000  13.0000  13.0000
646   Default   426.000000        people   426.000000  12.0000  12.0000
26    Default   202.000000           ask   202.000000  11.0000  11.0000
608   Default   624.000000           old   624.000000  10.0000  10.0000
465   Default   606.000000          know   606.000000   9.0000   9.0000
122   Default   352.000000           car   352.000000   8.0000   8.0000
346   Default   256.000000           god   256.000000   7.0000   7.0000
195   Default   321.000000          dead   321.000000   6.0000   6.0000
582   Default   149.000000         music   149.000000   5.0000   5.0000
701   Default   154.000000         radio   154.000000   4.0000   4.0000
520   Default   375.000000          look   375.000000   3.0000   3.0000
233   Default   355.000000         drink   355.000000   2.0000   2.0000
219   Default   669.000000           don   669.000000   1.0000   1.0000
243    Topic1    38.831306           dry    40.002726   1.3701  -6.2522
630    Topic1    23.788732       panties    24.848793   1.3562  -6.7423
79     Topic1    22.848634         board    23.940233   1.3531  -6.7826
1027   Topic1    23.679108     years ago    24.835435   1.3521  -6.7469
619    Topic1    18.604931      overhead    19.750286   1.3401  -6.9880
316    Topic1    35.798173          form    38.173050   1.3356  -6.3336
835    Topic1    34.051147      stairway    37.009811   1.3165  -6.3836
487    Topic1    46.603103           leg    51.339684   1.3030  -6.0698
640    Topic1    26.236380         party    29.192068   1.2931  -6.6443
806    Topic1    29.581717          slow    33.316057   1.2809  -6.5243
668    Topic1    21.157730        plenty    23.865210   1.2794  -6.8595
423    Topic1    78.174175         horse    90.450106   1.2540  -5.5525
1011   Topic1    49.607797         worst    57.652550   1.2495  -6.0073
343    Topic1    36.405722          glad    42.920500   1.2352  -6.3167
320    Topic1    37.886256          free    44.843289   1.2312  -6.2769
...       ...          ...           ...          ...      ...      ...
694   Topic25     0.051320       quickly    20.998890   1.3255  -6.9413
716   Topic25     0.051269  refrigerator    20.9956